In [1]:
#import required libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from IPython.display import display, Image
import csv
import glob
import re
from linreg_nn import *
from sketch_dec import SketchRNNDecoder
from draw_utils import plot_stroke

In [2]:
sketch_vec = "owl_z.csv"    #shape 100,128
photo_vec = "photo_z.csv"   # shape 100,7,7,160
# photo_dir = "/home/kelvin/OgataLab/sketch-wmultiple-tags/rendered_256x256/256x256/photo/tx_000000000000/owl/"
ele_photo_dir = "/home/kelvin/OgataLab/sketch-wmultiple-tags/dataset/photos/elephant/"
pig_photo_dir = "/home/kelvin/OgataLab/sketch-wmultiple-tags/dataset/photos/pig/"
MODEL_DIR = "./linreg_log/duo_set/DEFAULT/"
# MODEL_DIR = "./linreg_log/duo_set/b20l1-e5E100k-3L20010050/"
# MODEL_DIR = "./linreg_log/test4"#log for pseudo inverse training

# inputs, targets = get_csv_dataset()
# test_in = inputs[-5:]
# test_tar = targets[-5:]
# inp,targ = test_in, test_tar
"""load from npz"""
npz_dir = "./dataset/"
elepig = np.load(npz_dir+"elephantpig.npz")
# test
inp = elepig['test_photo']
targ = elepig['test_sketch']

In [3]:
#load photos

# img_n = list(range(1,101))
# img_arr = []
# for i in range(len(img_n)):
#     img_arr.append(photo_dir+str(img_n[i])+'.jpg')
ele_img_arr = glob.glob(ele_photo_dir+'/*.jpg')
pig_img_arr = glob.glob(pig_photo_dir+'/*.jpg')

ele_img_arr = sorted(ele_img_arr, key=lambda x: int(re.search(r'(?<=_)\w+', x).group()))
pig_img_arr = sorted(pig_img_arr, key=lambda x: int(re.search(r'(?<=_)\w+', x).group()))

img_arr = ele_img_arr[9:14] + pig_img_arr[0:4]
tar_p = img_arr
# print(tar_p)
# for i in range(len(tar_p)):
#     display(Image(filename=tar_p[i]))

In [4]:
#function to show images horizontally
def HoriImages(img_array):
    fig = plt.figure(figsize=(20,10))
    file_no = len(img_array)
    for i in range(file_no):
        a=fig.add_subplot(1,file_no,i+1)
        image = plt.imread(img_array[i])
        plt.imshow(image)
        plt.axis('off')

In [5]:
#load predictions from linear model
my_feature_columns = [tf.feature_column.numeric_column(
        key="x",
        shape=[7 * 7 * 160]
    )]    

my_config = tf.estimator.RunConfig(model_dir=MODEL_DIR)

# create a linear regression model
model = tf.estimator.Estimator(
        model_fn=linreg_fn,
        params={
            'feature_columns': my_feature_columns,
        },
        config=my_config
    )

pred_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': inp},
        y=targ,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_epochs=1
    )
preds = model.predict(input_fn=pred_input_fn)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe4976f9450>, '_model_dir': './linreg_log/duo_set/DEFAULT/', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_save_summary_steps': 100, '_num_ps_replicas': 0}


In [6]:
#call decoder
decoder = SketchRNNDecoder("/home/kelvin/OgataLab/sketchrnn_pretrained/elephantpig/lstm_test/")

INFO:tensorflow:Model using cpu.
INFO:tensorflow:Input dropout mode = 0.
INFO:tensorflow:Output dropout mode = 0.
INFO:tensorflow:Recurrent dropout mode = 0.
INFO:tensorflow:Loading model /home/kelvin/OgataLab/sketchrnn_pretrained/elephantpig/lstm_test/vector.
INFO:tensorflow:Restoring parameters from /home/kelvin/OgataLab/sketchrnn_pretrained/elephantpig/lstm_test/vector


In [7]:
#place vectors into list
stroke_t = []
stroke_p = []
for i in range(len(targ)):
    stroke_t.append(decoder.draw_from_z(np.expand_dims(targ[i],0)))
for pred in preds:
    stroke_p.append(decoder.draw_from_z(np.expand_dims(pred['sketch_vector'],0)))

INFO:tensorflow:Restoring parameters from ./linreg_log/duo_set/DEFAULT/model.ckpt-100000


NotFoundError: Key dense_1/kernel not found in checkpoint
	 [[Node: save/RestoreV2_3 = RestoreV2[dtypes=[DT_DOUBLE], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2_3/tensor_names, save/RestoreV2_3/shape_and_slices)]]

Caused by op u'save/RestoreV2_3', defined at:
  File "/home/kelvin/miniconda2/envs/magenta/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/kelvin/miniconda2/envs/magenta/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/kelvin/miniconda2/envs/magenta/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kelvin/miniconda2/envs/magenta/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kelvin/miniconda2/envs/magenta/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloo

In [ ]:
#Reconstruction display definition
def reconst(row, col, strokes):
    fig = plt.figure(figsize=(20,10))
    gs = gridspec.GridSpec(row, col)
    c=0
    for i in range(row):
        for j in range(col):
            ax = fig.add_subplot(gs[i, j])
            plot_stroke(ax, strokes[c])
            c+=1
    plt.show()

In [ ]:
#Reconstruction results from targets
# for i in range(len(stroke_t)):
#     ax = plt.subplot(111)
#     plot_stroke(ax,stroke_t[i])
#     plt.show()
reconst(1,9,stroke_t)

In [ ]:
#Reconstruction results from predictions
print len(stroke_p)
reconst(1,9,stroke_p)

In [ ]:
HoriImages(tar_p)
print("Untrained Photo targets")
# fig = plt.figure()

In [ ]:
print("Reconstruction from targets")
reconst(1,9,stroke_t)
print("Reconstruction from predictions")
reconst(1,9,stroke_p)